In [68]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from ipynb.fs.full.Pipeline import label_feature_data_and_merge
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import time 
from imblearn.under_sampling import RandomUnderSampler

In [3]:
def load_cache_data_and_preprocess_it(refactored_file_path, non_refactored_file_path):
    
    refactored_data = pd.read_feather(refactored_file_path)
    refactored_data.set_index("index", inplace=True)
    
    non_refactored_data = pd.read_feather(non_refactored_file_path)
    non_refactored_data.set_index("index", inplace=True)
    
    data = label_feature_data_and_merge(refactored_data, non_refactored_data)
    y = data["refactored"]
    x = data.drop("refactored", axis=1)
    
    print(y.value_counts())
    
    return (x, y)

In [210]:
# PICK first refactoring metric
refactored_file_path = ''.join(['data/', ''.join(METHOD_LEVEL_REFACTORINGS[13].lower().split()), '.ftr'])
non_refactored_file_path = '/Users/sanjaydutt/Documents/CSC503_project/data/nonrefactoreddata.ftr'

In [211]:
(x, y) = load_cache_data_and_preprocess_it(refactored_file_path, non_refactored_file_path)

0    66803
1     1213
Name: refactored, dtype: int64


In [212]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_over, y_over = undersample.fit_resample(x, y)

In [213]:
(X_train, X_test, y_train, y_test) = train_test_split(X_over, y_over, test_size=0.20, random_state=42)

In [214]:
X_train.shape

(1940, 69)

In [215]:
model = Sequential()
model.add(Dense(128, input_dim=69, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [216]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [217]:
t0 = time.time()
model.fit(X_train, y_train, epochs=20, batch_size=10)
print("Total time taken:", time.time() - t0)

Epoch 1/20
194/194 [==============================] - 1s 677us/step - loss: 38.2001 - accuracy: 0.6757
Epoch 2/20
194/194 [==============================] - 0s 644us/step - loss: 17.1503 - accuracy: 0.7232
Epoch 3/20
194/194 [==============================] - 0s 638us/step - loss: 9.6130 - accuracy: 0.7593
Epoch 4/20
194/194 [==============================] - 0s 684us/step - loss: 9.4271 - accuracy: 0.7732
Epoch 5/20
194/194 [==============================] - 0s 674us/step - loss: 6.9667 - accuracy: 0.7998
Epoch 6/20
194/194 [==============================] - 0s 650us/step - loss: 2.5669 - accuracy: 0.8228
Epoch 7/20
194/194 [==============================] - 0s 668us/step - loss: 3.7386 - accuracy: 0.8260
Epoch 8/20
194/194 [==============================] - 0s 656us/step - loss: 2.4851 - accuracy: 0.8376
Epoch 9/20
194/194 [==============================] - 0s 651us/step - loss: 2.0403 - accuracy: 0.8184
Epoch 10/20
194/194 [==============================] - 0s 742us/step - loss: 1.0

In [219]:
y_pred = np.round(model.predict(X_test),0)
print("Precision:",precision_score(y_test, y_pred))
print("Recall:",recall_score(y_test, y_pred))

Precision: 0.8617886178861789
Recall: 0.8945147679324894


In [220]:
t1 = time.time()
model.predict(X_test)
print("Training time:", time.time() - t1)

Training time: 0.04994487762451172
